
# Proyecto: Agente AI para Análisis de Noticias Financieras

**Instrucciones de uso:**

1. **Propósito:**
   - Este notebook analiza noticias financieras usando agentes AI (CrewAI/OpenAI y Zephyr), extrayendo entidades (empresas y países), generando resúmenes y clasificando el sentimiento.

2. **Carga de artículos:**
   - Puedes cargar archivos `.txt` manualmente o procesar todos los archivos de una carpeta por lotes.
   - Para descargar noticias automáticamente, configura tu clave de Newsdata.io y usa la función opcional.

3. **Procesamiento:**
   - El pipeline extrae entidades (solo empresas y países), genera un resumen relevante para cada entidad y clasifica el sentimiento como positivo, neutral o negativo.
   - Puedes comparar resultados entre modelos (OpenAI y Zephyr).

4. **Visualización y resultados:**
   - Los resultados se muestran en tablas y gráficos interactivos.
   - Si colocas archivos de referencia en la carpeta `ground_truths/`, las métricas de desempeño (precisión, recall, F1, ROUGE) se calculan automáticamente.

5. **Requisitos:**
   - Necesitas claves de API para OpenAI (y opcionalmente Newsdata.io).
   - Instala dependencias si es necesario (el notebook lo intenta automáticamente en Colab).

6. **Personalización:**
   - Puedes adaptar fácilmente el flujo para analizar un solo artículo o varios en lote.
   - Opcionalmente, puedes añadir tus propios módulos de análisis o visualización.

**Instalación y dependencias**

In [1]:
# Instala dependencias necesarias (solo la primera vez)
!pip install --quiet crewai openai tiktoken langchain_openai transformers spacy pandas pycountry textblob ipywidgets

**Importaciones y utilidades generales**

In [2]:
import os
import re
import json
import pandas as pd
from IPython.display import display, Markdown, HTML

**Configuración de claves y variables**

In [3]:
try:
    from google.colab import userdata, files
    os.environ["OPENAI_API_KEY"] = userdata.get('whitehole_key')
    os.environ["NEWSDATA_API_KEY"] = userdata.get('newsdata_key')
except ImportError:
    print("No se detectó entorno Colab. Configura tus claves manualmente si es necesario.")

**Utilidades generales**


In [4]:
# Función para cargar JSON desde string, lista o dict
# Devuelve lista o dict según lo esperado, o vacío si hay error de parseo
# Se usa para manejar salidas de modelos que pueden variar de formato
def json_loads(s, expect_list=False):
    import re
    try:
        obj = json.loads(s)
        if expect_list and not isinstance(obj, list):
            match = re.search(r'(\[.*?\])', s, re.DOTALL)
            if match:
                return json.loads(match.group(1))
            return []
        return obj
    except Exception:
        pattern = r'(\[.*?\])' if expect_list else r'(\{.*?\})'
        match = re.search(pattern, s, re.DOTALL)
        if match:
            try:
                return json.loads(match.group(1))
            except Exception:
                return [] if expect_list else {}
        return [] if expect_list else {}

# Extrae el resultado relevante de la salida de CrewAI
# Soporta varios formatos posibles: objeto con .result, .output, .raw, string JSON, lista, dict
# Devuelve el resultado parseado o lista vacía si falla
def parse_crew_output(output):
    if hasattr(output, 'result'):
        return output.result
    if hasattr(output, 'output'):
        return output.output
    if hasattr(output, 'raw'):
        try:
            return json.loads(output.raw)
        except Exception:
            return []
    if isinstance(output, (list, dict)):
        return output
    if isinstance(output, str):
        try:
            return json.loads(output)
        except Exception:
            return []
    return []

**Carga de articulo**

In [5]:
def cargar_articulo():
    try:
        uploaded = files.upload()  # Abre diálogo para subir archivo
        if not uploaded:
            raise RuntimeError('Debes subir al menos un archivo .txt.')
        filename = next(iter(uploaded))
        article_text = uploaded[filename].decode('utf-8')
        print(f'\nExtracto del artículo cargado ({filename}):\n')
        print(article_text[:500])  # Muestra los primeros 500 caracteres
    except Exception:
        raise RuntimeError('Error al cargar el archivo. Asegúrate de estar en Colab o implementa tu propio loader.')
    # Busca la fecha en el texto usando varias expresiones regulares
    date_patterns = [
        r'(20\d{2}[-/\.]\d{1,2}[-/\.]\d{1,2})',
        r'(\d{1,2}[-/\.]\d{1,2}[-/\.]20\d{2})',
        r'([A-Za-z]{3,9} \d{1,2},? 20\d{2})',
        r'(\d{1,2} [A-Za-z]{3,9} 20\d{2})',
        r'([A-Za-z]{3,9} \d{1,2},? \d{4})',
        r'(\d{1,2} de [A-Za-záéíóúñ]+ de 20\d{2})',
        r'(20\d{2})'
    ]
    article_date = "N/A"
    for pattern in date_patterns:
        match = re.search(pattern, article_text, re.IGNORECASE)
        if match:
            article_date = match.group(1)
            break
    print(f"\nFecha extraída del artículo: {article_date}")
    return article_text, article_date

**Pipelines de análisis**

In [6]:
# ---- CREWAI (OPENAI) ----
def pipeline_analisis_crewai(article_text, article_date):
    from crewai import Agent, Task, Crew, Process
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0.2)
    # Agentes
    extractor = Agent(
        role='Entity Extractor',
        goal='Identify companies and countries mentioned in the text.',
        backstory='You are a financial NLP expert. You only extract company and country names.',
        llm=llm
    )
    summarizer = Agent(
        role='Entity Summarizer',
        goal='Summarize relevant information about each extracted entity.',
        backstory='You are a financial analyst who summarizes key information about entities.',
        llm=llm
    )
    sentimenter = Agent(
        role='Sentiment Analyzer',
        goal='Classify the tone of the entity summary as positive, neutral, or negative.',
        backstory='You are an expert in financial sentiment analysis.',
        llm=llm
    )
    # Extracción
    task_extract = Task(
        description=(
            "Extract only companies and countries mentioned in the text. "
            "Normalize country names to their full official form (e.g., 'United States' for US/U.S./USA). "
            "Do not include cities, states, regions, people, central banks, organizations, or any other entity types. "
            "Return a JSON list of objects, each with 'entity' and 'type' fields. "
            "The 'type' field must be exactly 'company' or 'country'. "
            f"{article_text}"
        ),
        agent=extractor,
        expected_output='List of normalized entities (countries as full names, only parent companies) found in the text.'
    )
    crew_extract = Crew(
        agents=[extractor],
        tasks=[task_extract],
        process=Process.sequential
    )
    entities = parse_crew_output(crew_extract.kickoff())
    entities = [e for e in entities if isinstance(e, dict) and 'entity' in e and 'type' in e and e['type'] in ['company', 'country']]
    # Resumen y sentimiento
    results = []
    for ent in entities:
        entity = ent['entity']
        entity_type = ent['type']
        # Resumen
        task_summary = Task(
            description=(
                f"Given the following text:\n\n{article_text}\n\n"
                f"Generate a concise and relevant summary about the mention of entity '{entity}' (type: {entity_type}). "
                "Return only a JSON in the format: {\"entity\": ..., \"type\": ..., \"summary\": ...}, without any additional text or explanations."
            ),
            agent=summarizer,
            expected_output='Per-entity summary (with type).'
        )
        crew_summary = Crew(
            agents=[summarizer],
            tasks=[task_summary],
            process=Process.sequential
        )
        summary = parse_crew_output(crew_summary.kickoff())
        if not (isinstance(summary, dict) and 'entity' in summary and 'type' in summary and 'summary' in summary):
            summary = {"entity": entity, "type": entity_type, "summary": ""}
        # Sentimiento
        task_sentiment = Task(
            description=(
                f"For the following entity summary:\n\n{json.dumps(summary, ensure_ascii=False)}\n\n"
                "Classify the sentiment as 'positive', 'neutral', or 'negative'."
                "Return only a JSON in the format: {\"entity\": ..., \"type\": ..., \"sentiment\": ...}, without any additional text."
            ),
            agent=sentimenter,
            expected_output='Sentiment classification per entity (with type).'
        )
        crew_sentiment = Crew(
            agents=[sentimenter],
            tasks=[task_sentiment],
            process=Process.sequential
        )
        sentiment_json = parse_crew_output(crew_sentiment.kickoff())
        sentiment = sentiment_json.get("sentiment", "") if isinstance(sentiment_json, dict) else ""
        results.append({
            "date": article_date,
            "entity": summary.get("entity"),
            "type": summary.get("type"),
            "summary": summary.get("summary"),
            "sentiment": sentiment
        })
    return results

In [7]:
# ---- ZEPHYR (OPEN-SOURCE) ----
def pipeline_analisis_zephyr(article_text, article_date):
    import json
    import torch
    from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline as hf_pipeline

    # Carga el modelo solo si no está ya en memoria
    global zephyr_model, zephyr_tokenizer, zephyr_pipe
    if 'zephyr_model' not in globals():
        zephyr_model = AutoModelForCausalLM.from_pretrained(
            "HuggingFaceH4/zephyr-7b-beta",
            torch_dtype="auto",
            device_map="auto"
        )
    if 'zephyr_tokenizer' not in globals():
        zephyr_tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
    if 'zephyr_pipe' not in globals():
        zephyr_pipe = hf_pipeline(
            "text-generation",
            model=zephyr_model,
            tokenizer=zephyr_tokenizer,
            max_new_tokens=512,
            temperature=0.2,
            do_sample=False
        )

    def zephyr_inference(prompt):
        result = zephyr_pipe(prompt)
        return result[0]['generated_text'][len(prompt):].strip()

    # 1. Extracción de entidades
    prompt_extract = (
        "Extract only companies and countries mentioned in the text. "
        "Normalize country names to their full official form (e.g., 'United States' for US/U.S./USA). "
        "Do not include cities, states, regions, people, central banks, organizations, or any other entity types. "
        "Return a JSON list of objects, each with 'entity' and 'type' fields. "
        "The 'type' field must be exactly 'company' or 'country'. "
        f"{article_text}"
    )
    entities_str = zephyr_inference(prompt_extract)
    entities = json_loads(entities_str, expect_list=True)
    entities = [e for e in entities if isinstance(e, dict) and 'entity' in e and 'type' in e and e['type'] in ['company', 'country']]

    # 2. Resumen y sentimiento en una sola llamada por entidad
    results = []
    for ent in entities:
        entity = ent['entity']
        entity_type = ent['type']
        prompt_summary_sentiment = (
            f"Given the following text:\n\n{article_text}\n\n"
            f"Generate a concise and relevant summary about the mention of entity '{entity}' (type: {entity_type}). "
            "Return only a JSON in the format: {\"entity\": ..., \"type\": ..., \"summary\": ...}, without any additional text or explanations."
        )
        summary_sentiment_str = zephyr_inference(prompt_summary_sentiment)
        summary_sentiment = json_loads(summary_sentiment_str)
        # Validación y fallback
        if not (isinstance(summary_sentiment, dict) and all(k in summary_sentiment for k in ["entity", "type", "summary", "sentiment"])):
            summary_sentiment = {"entity": entity, "type": entity_type, "summary": "", "sentiment": ""}
        results.append({
            "date": article_date,
            "entity": summary_sentiment.get("entity", entity),
            "type": summary_sentiment.get("type", entity_type),
            "summary": summary_sentiment.get("summary", ""),
            "sentiment": summary_sentiment.get("sentiment", "")
        })
    return results

**Visualización y exploración interactiva**

In [8]:
def mostrar_resultados(df, titulo):
    print(f'=== {titulo} ===')
    display(df)

def show_entity_explorer(df_widgets):
    import plotly.express as px
    import ipywidgets as widgets
    from IPython.display import display, Markdown, HTML, clear_output
    if df_widgets.empty or not df_widgets['entity'].iloc[0]:
        display(Markdown('**No hay entidades para mostrar.**'))
        return
    # Gráficos
    fig1 = px.bar(
        df_widgets.groupby('type').size().reset_index(name='count'),
        x='type', y='count', color='type',
        title='Número de entidades por tipo',
        color_discrete_sequence=px.colors.qualitative.Safe
    )
    fig2 = px.histogram(
        df_widgets, x='entity', color='sentiment',
        title='Sentimiento por entidad',
        color_discrete_map={'positivo':'#1976d2','negativo':'#d32f2f','neutral':'#ffa000'},
        category_orders={'sentiment': ['positivo', 'neutral', 'negativo']}
    )
    display(HTML('<h3 style="color:#1565c0; margin-bottom:10px;">Visualización de entidades</h3>'))
    fig1.show()
    fig2.show()
    # Interactivo
    entity_options = df_widgets['entity'].unique().tolist()
    dropdown = widgets.Dropdown(
        options=entity_options,
        description='Entidad:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='50%', margin='0 0 20px 0')
    )
    output = widgets.Output()

    def on_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            output.clear_output()
            selected = change['new']
            info = df_widgets[df_widgets['entity'] == selected].iloc[0]
            bias = info.get('bias_flag', '')
            bias_html = f'<tr><td style="font-weight:bold; color:#b71c1c;">Bias/Framing:</td><td style="color:#b71c1c;">{bias}</td></tr>' if bias else ''
            with output:
                display(HTML(f'''
                    <div style="border:1px solid #1976d2; border-radius:12px; padding:22px; background:#f6fafd; box-shadow: 0 2px 8px #e3e9f2; max-width:700px;">
                        <h3 style="margin-top:0; color:#1976d2;">{info['entity']}</h3>
                        <table style="width:100%; border-collapse:collapse;">
                          <tr>
                            <td style="font-weight:bold; color:#444; width:120px;">Tipo:</td>
                            <td style="color:#222;">{info['type']}</td>
                          </tr>
                          <tr>
                            <td style="font-weight:bold; color:#444;">Fecha:</td>
                            <td style="color:#222;">{info['date']}</td>
                          </tr>
                          <tr>
                            <td style="font-weight:bold; color:#444; vertical-align:top;">Resumen:</td>
                            <td style="color:#222; background:#f0f4f8; padding:10px; border-radius:7px;">{info['summary']}</td>
                          </tr>
                          <tr>
                            <td style="font-weight:bold; color:#444;">Sentimiento:</td>
                            <td style="color:#1976d2; font-weight:bold;">{info['sentiment'].capitalize()}</td>
                          </tr>
                          {bias_html}
                        </table>
                    </div>
                '''))

    dropdown.observe(on_change)
    display(dropdown)
    display(output)
    if entity_options:
        dropdown.value = entity_options[0]

**Descarga NewsData.io y bias**

In [9]:
def download_newsdata_articles(api_key, query, language="es", country=None, category=None, max_articles=10, save_dir="newsdata_articles"):
    import requests, os
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    url = "https://newsdata.io/api/1/news"
    params = {
        "apikey": api_key,
        "q": query,
        "language": language,
        "country": country,
        "category": category
    }
    articles_downloaded = 0
    next_page = None
    while articles_downloaded < max_articles:
        req_params = params.copy()
        if next_page:
            req_params["page"] = next_page
        resp = requests.get(url, params={k: v for k, v in req_params.items() if v})
        if resp.status_code != 200:
            print(f"Error {resp.status_code}: {resp.text}")
            break
        data = resp.json()
        results = data.get("results", [])
        if not results:
            break
        for art in results:
            if articles_downloaded >= max_articles:
                break
            title = art.get("title", "untitled").replace("/", "_")[:80]
            content = art.get("content") or art.get("description") or art.get("full_content") or ""
            pub_date = art.get("pubDate", "").replace(":", "-").replace("T", "_")[:16]
            fname = f"{pub_date}_{title}.txt"
            fpath = os.path.join(save_dir, fname)
            with open(fpath, "w", encoding="utf-8") as f:
                f.write(content)
            articles_downloaded += 1
        next_page = data.get("nextPage")
        if not next_page:
            break
    print(f"Descargados {articles_downloaded} artículos en '{save_dir}'")

def aplicar_bias_textblob(df):
    try:
        from textblob import TextBlob
    except ImportError:
        print('TextBlob no está instalado. No se puede analizar bias.')
        return df
    def detect_bias_textblob(row):
        # Solo buscar sesgo si el sentimiento original es neutral
        if row['sentiment'].lower() != 'neutral':
            return ''
        summary = row['summary']
        blob = TextBlob(summary)
        polarity = blob.sentiment.polarity
        # Umbrales heurísticos
        if polarity > 0.15:
            return 'Neutral (sesgo positivo detectado por TextBlob)'
        elif polarity < -0.15:
            return 'Neutral (sesgo negativo detectado por TextBlob)'
        else:
            return ''
    df = df.copy()
    df['bias_flag'] = df.apply(detect_bias_textblob, axis=1)
    return df

**Utilidades para tokens, coste y lote**

In [10]:
def count_tokens_openai(prompt, model='gpt-4.1-nano'):
    try:
        import tiktoken
        enc = tiktoken.encoding_for_model(model)
        return len(enc.encode(prompt))
    except Exception:
        return -1

def summarize_efficiency_metrics(metrics, token_counts=None, price_per_1k=0.01):
    import pandas as pd
    from IPython.display import display, Markdown
    df = pd.DataFrame(metrics)
    display(Markdown('### Métricas de eficiencia por artículo:'))
    display(df)
    if token_counts is not None:
        total_tokens = sum(token_counts)
        coste = total_tokens / 1000 * price_per_1k
        display(Markdown(f'- **Total tokens:** {total_tokens}'))
        display(Markdown(f'- **Coste estimado:** ${coste:.4f}'))
        display(Markdown(f'- **Llamadas API:** {len(token_counts)}'))

def process_articles_with_pipeline(articles, pipeline_func):
    import time
    outputs = []
    metrics = []
    token_counts = []
    for article_text, fname in articles:
        t0 = time.time()
        output = pipeline_func(article_text, fname)
        t1 = time.time()
        # Token count (solo para OpenAI)
        try:
            prompt_len = count_tokens_openai(article_text)
        except Exception:
            prompt_len = -1
        token_counts.append(prompt_len)
        metrics.append({'file': fname, 'time': t1-t0, 'n_entities': len(output), 'tokens': prompt_len})
        outputs.append(output)
    return outputs, metrics, token_counts

**Carga y procesamiento en lote (multiartículo)**

In [11]:
def load_all_txt_articles(folder_path='.'):
    import glob
    files = glob.glob(f'{folder_path}/*.txt')
    articles = []
    for fname in files:
        with open(fname, 'r', encoding='utf-8') as f:
            articles.append((f.read(), fname))
    return articles

def combine_outputs_to_df(outputs):
    import pandas as pd
    all_rows = [row for output in outputs for row in output]
    return pd.DataFrame(all_rows)

**Flujo principal OpenAI**

* Para utilizar la descarga de archivos desde NewsData, tienes que descomentar el punto 5 en esta misma celda.
* Para utilizar el flujo de Zephyr, tienes que descomentar el punto 6 en esta misma celda.

In [12]:
if __name__ == "__main__":
    # 1. Cargar artículo(s) y fecha
    modo_lote = False # Cambia a True para procesar todos los .txt de una carpeta
    if not modo_lote:
        article_text, article_date = cargar_articulo()
        # 2. Analizar con CrewAI (OpenAI)
        output_gpt = pipeline_analisis_crewai(article_text, article_date)
        df_gpt = pd.DataFrame(output_gpt)
        mostrar_resultados(df_gpt, 'Resultados CrewAI (GPT-4.1-nano)')
        # 3. Bias (opcional)
        try:
            df_gpt = aplicar_bias_textblob(df_gpt)
            mostrar_resultados(df_gpt, 'Resultados CrewAI + Bias')
        except Exception:
            pass
        # 4. Visualización interactiva
        try:
            show_entity_explorer(df_gpt)
        except Exception:
            print('Visualización interactiva no disponible.')
        # 5. Descarga de noticias (opcional)
        # download_newsdata_articles(os.environ.get('NEWSDATA_API_KEY'), query='economía', language='es', max_articles=10)
        # 6. Análisis con Zephyr (opcional, separado)
        # Para usar Zephyr, descomenta la siguiente línea:
        # analizar_con_zephyr(article_text, article_date)
    else:
        # Procesamiento en lote
        folder = './newsdata_articles' # Cambia a la ruta de tus artículos
        articles = load_all_txt_articles(folder)
        outputs_gpt, metrics_gpt, token_counts = process_articles_with_pipeline(articles, lambda text, fname: pipeline_analisis_crewai(text, 'N/A'))
        df_gpt_all = combine_outputs_to_df(outputs_gpt)
        mostrar_resultados(df_gpt_all, 'Resultados CrewAI (GPT-4.1-nano) - Lote')
        summarize_efficiency_metrics(metrics_gpt, token_counts)
        # Visualización temporal por entidad (opcional)
        # plot_all_entities_sentiment_timeseries(df_gpt_all)
        # --- Evaluación automática de desempeño ---
        # (métricas automáticas si hay ground truth)
        gt_entities = load_ground_truth_entities() if 'load_ground_truth_entities' in globals() else {}
        gt_sentiment = load_ground_truth_sentiment() if 'load_ground_truth_sentiment' in globals() else {}
        gt_summaries = load_ground_truth_summaries() if 'load_ground_truth_summaries' in globals() else {}
        if gt_entities:
            evaluate_entity_extraction(df_gpt_all, gt_entities)
        if gt_sentiment:
            evaluate_sentiment_classification(df_gpt_all, gt_sentiment)
        if gt_summaries:
            evaluate_summaries_rouge(df_gpt_all, gt_summaries)

Saving news_02.txt to news_02.txt

Extracto del artículo cargado (news_02.txt):

Week ahead: Eurozone inflation, Apple and Meta earnings in focus

Week ahead: Eurozone inflation, Apple and Meta earnings in focus · Euronews
Tina Teng
Mon, April 28, 2025 at 9:15 AM GMT+2 4 min read

Global markets rebounded last week on a broad-based rally amid signs of de-escalation in the US-China trade war. Investors will continue to monitor major economic data this week, including the eurozone’s monthly inflation figures and the United States’ jobs report.

Additionally, major US technolog

Fecha extraída del artículo: April 28, 2025
=== Resultados CrewAI (GPT-4.1-nano) ===


,date,entity,type,summary,sentiment
0,"April 28, 2025",Eurozone,country,The Eurozone is scheduled to release its flash...,neutral
1,"April 28, 2025",United States,country,"In the US, key economic indicators to be relea...",neutral
2,"April 28, 2025",Germany,country,Consumer prices declined across most major Eur...,neutral
3,"April 28, 2025",Spain,country,Consumer prices declined across most major Eur...,positive
4,"April 28, 2025",Netherlands,country,Consumer prices declined across most major Eur...,positive
5,"April 28, 2025",Belgium,country,Consumer prices declined across most major Eur...,neutral
6,"April 28, 2025",France,country,"Inflation in France remained steady in March, ...",neutral
7,"April 28, 2025",Italy,country,"Inflation in Italy accelerated in March, and t...",neutral
8,"April 28, 2025",China,country,Investors are monitoring China's manufacturing...,neutral
9,"April 28, 2025",India,country,Apple reportedly plans to shift all US-sold iP...,positive


=== Resultados CrewAI + Bias ===


,date,entity,type,summary,sentiment,bias_flag
0,"April 28, 2025",Eurozone,country,The Eurozone is scheduled to release its flash...,neutral,
1,"April 28, 2025",United States,country,"In the US, key economic indicators to be relea...",neutral,
2,"April 28, 2025",Germany,country,Consumer prices declined across most major Eur...,neutral,Neutral (sesgo positivo detectado por TextBlob)
3,"April 28, 2025",Spain,country,Consumer prices declined across most major Eur...,positive,
4,"April 28, 2025",Netherlands,country,Consumer prices declined across most major Eur...,positive,
5,"April 28, 2025",Belgium,country,Consumer prices declined across most major Eur...,neutral,Neutral (sesgo positivo detectado por TextBlob)
6,"April 28, 2025",France,country,"Inflation in France remained steady in March, ...",neutral,
7,"April 28, 2025",Italy,country,"Inflation in Italy accelerated in March, and t...",neutral,
8,"April 28, 2025",China,country,Investors are monitoring China's manufacturing...,neutral,Neutral (sesgo positivo detectado por TextBlob)
9,"April 28, 2025",India,country,Apple reportedly plans to shift all US-sold iP...,positive,


Dropdown(description='Entidad:', layout=Layout(margin='0 0 20px 0', width='50%'), options=('Eurozone', 'United…

Output()

**Comparación CrewAI vs Zephyr**

 Ejecuta este apartado después de analizar un artículo con CrewAI para comparar resultados con Zephyr.



 1. Asegúrate de haber ejecutado el flujo principal y tener `article_text` y `article_date` cargados.

 2. Ejecuta este apartado para obtener y mostrar los resultados de Zephyr junto a los de CrewAI.

In [13]:
# Ejecuta análisis con Zephyr (opcional)
df_zephyr = pipeline_analisis_zephyr(article_text, article_date)

# Comparar resultados (si ambos existen)
if 'df_gpt' in globals() and df_zephyr is not None:
    from IPython.display import display, Markdown
    display(Markdown('### Comparativa de entidades extraídas'))
    entidades_gpt = set(df_gpt['entity'])
    entidades_zephyr = set(df_zephyr['entity'])
    display(Markdown(f"**CrewAI:** {entidades_gpt}"))
    display(Markdown(f"**Zephyr:** {entidades_zephyr}"))

    display(Markdown('### Comparativa de sentimiento por entidad'))
    df_comp = df_gpt[['entity', 'sentiment']].merge(
        df_zephyr[['entity', 'sentiment']], on='entity', how='outer', suffixes=('_gpt', '_zephyr')
    )
    display(df_comp)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not in

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning:

`do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.



KeyboardInterrupt: 